Useful Libraries

In [1]:
import pandas as pd
import numpy as np

Let's open the CSV created in Scrapping.ipyng

In [2]:
actions = pd.read_csv('actions.csv')
actions.head()

,asso,title,url,all_text,summary,key_words
0,Secours Populaire,Urgences climatiques,https://www.secourspopulaire.fr/nos-actions/ur...,Protéger Dès lors qu’une catastrophe climatiqu...,(Source : Haut Commissariat de l’ONU pour les...,"['populaire', 'secours populaire', 'secours', ..."
1,Secours Populaire,Conflits,https://www.secourspopulaire.fr/nos-actions/co...,"Protéger Face aux conflits armés, l’entraide e...","Distribution de colis alimentaires, de kits d...","['vie', 'urgence', 'secours', 'secours populai..."
2,Secours Populaire,Aide aux migrants-réfugiés,https://www.secourspopulaire.fr/nos-actions/ai...,"Protéger Chaque jour, des familles, des enfant...",Le Secours populaire se mobilise pour les aid...,"['aide', 'secours', 'bénévoles', 'populaire', ..."
3,Secours Populaire,Aide au logement,https://www.secourspopulaire.fr/nos-actions/ai...,"Protéger Dans les permanences d'accueil, de so...","Protéger Dans les permanences d'accueil, de s...","['logement', 'populaire', 'plus', 'personnes',..."
4,Secours Populaire,Accès aux vacances,https://www.secourspopulaire.fr/nos-actions/ac...,"Émanciper Les vacances, essentielles, sont de ...","Pourtant, aujourd’hui encore, un enfant sur t...","['vacances', 'plus', 'familles', 'populaire', ..."


In [3]:
actions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   asso       27 non-null     object
 1   title      27 non-null     object
 2   url        27 non-null     object
 3   all_text   27 non-null     object
 4   summary    27 non-null     object
 5   key_words  27 non-null     object
dtypes: object(6)
memory usage: 1.4+ KB


In [4]:
actions.isna().sum()

asso         0
title        0
url          0
all_text     0
summary      0
key_words    0
dtype: int64

Lets try a first summary function

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

def summarize_text2(text):
    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text)

    freqTable = dict()
    for word in words:
        word = word.lower()
        if word not in stopWords:
            if word in freqTable:
                freqTable[word] += 1
            else:
                freqTable[word] = 1

    sentences = sent_tokenize(text)
    sentenceValue = dict()

    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq

    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]

    # Average value of a sentence from the original text
    average = int(sumValues / len(sentenceValue))

    summary = ''
    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
            summary += " " + sentence

    return summary

# Trying our function
text = actions.iloc[2]['all_text']
print(summarize_text2(text))


 Le Secours populaire se mobilise pour les aider à reconstruire leur vie, d’abord avec une aide d’urgence, puis en les accompagnant dans leur insertion sociale. Les migrants, les réfugiés, ce sont des femmes, des hommes, des enfants qui fuient la misère, la guerre, un climat hostile et se retrouvent avec une vie à réinventer. Aide alimentaire, kits d’hygiène, vêtements, mise à l’abri… Le Secours populaire leur apporte l’aide d’urgence et le réconfort dont ils ont tant besoin. En France, comme dans tous les pays où ils ont trouvé refuge, le Secours populaire et ses partenaires dans le monde ont à cœur d’apporter une aide à la mesure d’un défi mondial. Soutien psychologique, cours de français, accès aux droits, sorties culturelles et vacances… Les bénévoles les accompagnent dans la durée pour les aider à reprendre le cours de leur vie. La chaîne de solidarité est telle que de nombreuses personnes migrantes et réfugiées s’engagent à leur tour en tant que bénévoles au Secours populaire. Au

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nberl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nberl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The summary is a little long, but the content is satisfactory. Let's try to limit the number of characters.

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')

def summarize_text(text, max_length=500):  # Adding a max length parameter to the summary
    stopWords = set(stopwords.words("french"))
    words = word_tokenize(text)

    freqTable = dict()
    for word in words:
        word = word.lower()
        if word not in stopWords:
            if word in freqTable:
                freqTable[word] += 1
            else:
                freqTable[word] = 1

    sentences = sent_tokenize(text)
    sentenceValue = dict()

    for sentence in sentences:
        for word, freq in freqTable.items():
            if word in sentence.lower():
                if sentence in sentenceValue:
                    sentenceValue[sentence] += freq
                else:
                    sentenceValue[sentence] = freq

    sumValues = 0
    for sentence in sentenceValue:
        sumValues += sentenceValue[sentence]

    average = int(sumValues / len(sentenceValue))

    summary = ''
    for sentence in sentences:
        if (sentence in sentenceValue) and (sentenceValue[sentence] > (1.2 * average)):
            if len(summary) + len(sentence) <= max_length:  # Verify if the summary is still under the max length
                summary += " " + sentence
            else:
                break  # Stop the loop if the summary is too long

    return summary

# Example
text = actions.iloc[2]['all_text']
print(summarize_text(text, 500))  # Specifying the max length


 Le Secours populaire se mobilise pour les aider à reconstruire leur vie, d’abord avec une aide d’urgence, puis en les accompagnant dans leur insertion sociale. Aide alimentaire, kits d’hygiène, vêtements, mise à l’abri… Le Secours populaire leur apporte l’aide d’urgence et le réconfort dont ils ont tant besoin. En France, comme dans tous les pays où ils ont trouvé refuge, le Secours populaire et ses partenaires dans le monde ont à cœur d’apporter une aide à la mesure d’un défi mondial.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nberl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nberl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


La limite de caractère est bien appliqué mais ne fait que couper les phrases du texte généré. Essayons une autre méthode

In [7]:
from transformers import BartTokenizer, BartForConditionalGeneration

def summarize_with_bart(text, model_name='facebook/bart-large-cnn', max_char=500):
    # Initialize tokenizer and model
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    # Tokenize and encode the text
    inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)

    # Model generate summary
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=max_char, early_stopping=True)

    # Decode and return the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

c:\Users\nberl\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
text = actions.iloc[2]['all_text']
print(summarize_with_bart(text))

Le Secours populaire mobilise pour les aider à reconstruire leur vie. L'aide déployée par les bénévoles est une réponse à un enjeu historique. Les migrants, les réfugiés, ce sont des femmes, des hommes, des enfants.


At first glance, the result is less satisfying than before.

Let's create a 'Summary' column in our DataFrame to try and improve BART summaries as a first step.

In [9]:
actions['summary'] = actions['all_text'].apply(summarize_with_bart)

Let's try a language tool

In [10]:
import language_tool_python

def check_grammar_french(text):
    # Creating a LanguageTool instance for French since our data is written in French
    tool = language_tool_python.LanguageTool('fr')

    # Checking grammar
    matches = tool.check(text)

    # Extract and display suggestions
    for match in matches:
        print(f"Issue: {match.ruleId}, Message: {match.message}")

    # We can also obtain a corrected version of the text
    corrected_text = tool.correct(text)
    return corrected_text

# Example
text = "Votre texte en français ici."
corrected_text = check_grammar_french(text)
print("\nTexte corrigé :")
print(corrected_text)



Texte corrigé :
Votre texte en français ici.


In [11]:
text = actions.iloc[2]['summary']
print(check_grammar_french(text))

Le Secours populaire mobilise pour les aider à reconstruire leur vie. L'aide déployée par les bénévoles est une réponse à un enjeu historique. Les migrants, les réfugiés, ce sont des femmes, des hommes, des enfants.


The result has not really improved. So we'll keep the summaries from our first function.

In [12]:
actions['summary'] = actions['all_text'].apply(summarize_text, max_length=500)

In [13]:
actions.head()

,asso,title,url,all_text,summary,key_words
0,Secours Populaire,Urgences climatiques,https://www.secourspopulaire.fr/nos-actions/ur...,Protéger Dès lors qu’une catastrophe climatiqu...,(Source : Haut Commissariat de l’ONU pour les...,"['populaire', 'secours populaire', 'secours', ..."
1,Secours Populaire,Conflits,https://www.secourspopulaire.fr/nos-actions/co...,"Protéger Face aux conflits armés, l’entraide e...","Distribution de colis alimentaires, de kits d...","['vie', 'urgence', 'secours', 'secours populai..."
2,Secours Populaire,Aide aux migrants-réfugiés,https://www.secourspopulaire.fr/nos-actions/ai...,"Protéger Chaque jour, des familles, des enfant...",Le Secours populaire se mobilise pour les aid...,"['aide', 'secours', 'bénévoles', 'populaire', ..."
3,Secours Populaire,Aide au logement,https://www.secourspopulaire.fr/nos-actions/ai...,"Protéger Dans les permanences d'accueil, de so...","Protéger Dans les permanences d'accueil, de s...","['logement', 'populaire', 'plus', 'personnes',..."
4,Secours Populaire,Accès aux vacances,https://www.secourspopulaire.fr/nos-actions/ac...,"Émanciper Les vacances, essentielles, sont de ...","Pourtant, aujourd’hui encore, un enfant sur t...","['vacances', 'plus', 'familles', 'populaire', ..."


In addition to the summary, a search module based on a list of keywords would be ideal. Let's put it in place.

In [14]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

#nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words_french = list(stopwords.words('french'))  # Convert the NLTK list to a Python list

def extract_keywords_tfidf(text, num_keywords=15): #10 to 15 keywords should be good number
    # Use tf-idf vectorizer
    vectorizer = TfidfVectorizer(stop_words=stop_words_french, ngram_range=(1, 2))
    tfidf_matrix = vectorizer.fit_transform([text])
    feature_array = vectorizer.get_feature_names_out()
    tfidf_sorting = tfidf_matrix.toarray().flatten().argsort()[-num_keywords:]

    keywords = [feature_array[i] for i in tfidf_sorting[::-1]]
    
    return keywords

# Example
text = actions.iloc[1]['all_text']
print(extract_keywords_tfidf(text))


['vie', 'urgence', 'secours', 'secours populaire', 'personnes', 'populaire', 'pays', 'leurs', 'aider', 'aide', 'guerre', 'droits', 'soutien', 'permet', 'sorties']


In [15]:
actions['key_words'] = actions['all_text'].apply(extract_keywords_tfidf)

In [16]:
actions.loc[actions['summary'] == '', 'summary'] = actions.loc[actions['summary'] == '', 'all_text']


In [18]:
actions.iloc[0]['summary']

' (Source : Haut Commissariat de l’ONU pour les réfugiés) Inondations, tempêtes, fortes intempéries, grand froid, catastrophes naturelles ou industrielles, incendies… Les conséquences de ces drames sont plus terribles encore chez les familles en situation de pauvreté. Dès qu’un drame humanitaire se produit, un appel aux dons est lancé par le Secours populaire et des aides d’urgence sont apportées au plus vite aux sinistrés.'

In [38]:
actions.to_csv('actions.csv', index=False)

In [14]:
assos = pd.read_csv('assos.csv')
assos.head()

,nom,activites,lib_activité,nb_etablissements,url,adresses
0,SECOURS POPULAIRE FRANCAIS,soutenir matériellement moralement juridiqueme...,Action sociale sans hébergement n.c.a.,1,https://www.secourspopulaire.fr/,"[{'num_voie': 10, 'type_voie': 'RUE', 'voie': ..."
1,REGLES ELEMENTAIRES,aider les personnes les plus démunies à accéde...,Autres organisations fonctionnant par adhésion...,1,https://www.regleselementaires.com/,"[{'cplt_2': 'LA CITE AUDACIEUSE', 'num_voie': ..."
2,EMMAUS SOLIDARITE,développer des actions de solidarité partagées...,Hébergement social pour adultes et familles en...,11,https://www.emmaus-solidarite.org/,"[{'num_voie': 10, 'type_voie': 'RUE', 'voie': ..."


Now we need to process our addresses for the website

In [16]:
import ast

def format_address(adresse_str):
    try:
        # Convert string to dictionary
        adresses = ast.literal_eval(adresse_str)
    except ValueError:
        # Return an empty string if conversion fails
        return ''

    formatted_addresses = []
    for adresse in adresses:
        num_voie = adresse.get('num_voie', '')
        type_voie = adresse.get('type_voie', '')
        voie = adresse.get('voie', '')
        commune = adresse.get('commune', '')
        cp = adresse.get('cp', '')

        formatted_address = f"{num_voie} {type_voie} {voie}, {commune} {cp}".strip().lower()
        formatted_addresses.append(formatted_address)

    return formatted_addresses

# Applying the function to each DataFrame line
assos['adresses_processed'] = assos['adresses'].apply(format_address)


In [17]:
assos.head()

,nom,activites,lib_activité,nb_etablissements,url,adresses,adresses_processed
0,SECOURS POPULAIRE FRANCAIS,soutenir matériellement moralement juridiqueme...,Action sociale sans hébergement n.c.a.,1,https://www.secourspopulaire.fr/,"[{'num_voie': 10, 'type_voie': 'RUE', 'voie': ...","[10 rue paul bert, belfort 90000, 131 de pari..."
1,REGLES ELEMENTAIRES,aider les personnes les plus démunies à accéde...,Autres organisations fonctionnant par adhésion...,1,https://www.regleselementaires.com/,"[{'cplt_2': 'LA CITE AUDACIEUSE', 'num_voie': ...","[9 rue de vaugirard, paris 6 75006, 74 denfer..."
2,EMMAUS SOLIDARITE,développer des actions de solidarité partagées...,Hébergement social pour adultes et familles en...,11,https://www.emmaus-solidarite.org/,"[{'num_voie': 10, 'type_voie': 'RUE', 'voie': ...","[10 rue de la baignade, ivry-sur-seine 94200, ..."


In [18]:
assos.to_csv('assos.csv', index=False)